In [ ]:
# Imports
import os
import requests
from os import path
from bs4 import BeautifulSoup

In [ ]:
# Read file method
def read_file(filename):
    with open(filename) as f:
        content = f.readlines()
    return [x.strip() for x in content]

# Method to check if file exists or not
def checkfile(filename):
    return path.exists(filename)

In [ ]:
# Specify the path to the direcory where your 4 parameter files mentioned in the next cell are present.
parameter_path = 'parameter'

In [ ]:
# Pass the file names for the respective things, please don't change these names if possible, if you are changing, please be careful
hyperlink_file = "hyperlink.txt"
filenames_file = "savefiles.txt"
baselink_file = "base.txt"
parameter_file = "parameters.txt"

In [ ]:
# Change the years as per your requirement
years = ["2014-2015", "2015-2016", "2016-2017", "2017-2018", "2018-2019","2019-2020", "2020-2021"]

# reading all the files for specifications and getting the content into lists
hyperlink_lines = read_file(path.join(parameter_path, hyperlink_file))
parameter_lines = read_file(path.join(parameter_path, parameter_file))
savedhtml_lines = read_file(path.join(parameter_path, filenames_file))
basehtml_lines = read_file(path.join(parameter_path, baselink_file))

In [ ]:
for i in range(len(hyperlink_lines)):
    # Getting the url
    url = hyperlink_lines[i]

    # Getting the parameter name
    param = savedhtml_lines[i]

    # Creating the directory for the parameter, if doesn't exist
    param_dir = path.join("html", param)
    if not path.join(param_dir):
        os.makedirs(param_dir)

    # Iterating through the years
    for year in years:

        # Creating the directory for the yeawr, if doesn't exist
        year_dir = path.join(param_dir, year)
        if not path.exists(year_dir):
            os.makedirs(year_dir)

        # Appending the year parameter to the url
        year_url = url + year
        print(f'year url: {year_url}')
        soup = BeautifulSoup(requests.get(year_url).content,'html.parser')

        # Getting all the states into the table on the webpage
        states = soup.findAll("tr")[5:-1]

        # Iterating through each of the state
        for state in states:
            try:
                # State name with link in the 2nd column, so moving to the next sibling
                state_element = state.find('td').find_next_sibling("td")

                # Getting the href link from the element
                href_element = state_element.find_all(href = True)

                # Check if there exist an href link
                if len(href_element) != 0:
                    # Append the link to the base html url
                    html_link = basehtml_lines[i] + href_element[0]['href']

                    state_name = href_element[0].text
                    print(f'State: {state_name}')

                    # Check if the state name is non-empty
                    if state_name == "":
                        print(f"State not available")
                        continue

                    # Creating the directory for the state if doesn't exist
                    state_dir = path.join(year_dir, state_name)
                    if not path.exists(state_dir):
                        os.makedirs(state_dir)

                    # Getting the districts using the link for the state formed above
                    dsoup = BeautifulSoup(requests.get(html_link).content,'html.parser')
                    districts = dsoup.findAll("tr")[5:-1]

                    # Iterating over each of the district
                    for district in districts:
                        try:

                            # Getting the district link element from the 2nd column
                            district_element = district.find('td').find_next_sibling("td")

                            # Getting the href element from it
                            href_element = district_element.find_all(href=True)

                            # If there exist a link, go ahead
                            if len(href_element) != 0:

                                # Append the link to the base url                       
                                html_link = basehtml_lines[i] + href_element[0]['href']

                                district_name = href_element[0].text

                                # Check if the district name is non-empty
                                if district_name == "":
                                    print(f"District in State: {state_name} not available")

                                # Create the district directory if doesn't exist
                                district_dir = path.join(state_dir, district_name)
                                if not path.exists(district_dir):
                                    os.makedirs(district_dir)
                    
                                # Getting the blocks from the district page link formed above
                                bsoup = BeautifulSoup(requests.get(html_link).content,'html.parser')
                                blocks = bsoup.findAll("tr")[5:-1]

                                # Iterating through each of the block
                                for block in blocks:
                                    try:

                                        # Getting the block name and link from the 2nd column
                                        block_element = block.find('td').find_next_sibling("td")
                                        href_element = block_element.find_all(href=True)

                                        # Check if there exists a link
                                        if len(href_element) != 0:

                                            # Append the href link obtained to the base url
                                            html_link = basehtml_lines[i] + href_element[0]['href']
                                            block_name = href_element[0].text

                                            # Check if the block name is empty
                                            if block_name == "":
                                                print(f"Block in District {district_name} not available")
                                            
                                            # Check if the block file already exists
                                            if path.exists(path.join(district_dir, block_name + ".html")):
                                                continue

                                            # Getting the block page
                                            block_page = requests.get(html_link).content

                                            # Saving the block page to html file
                                            file = open(os.path.join(district_dir, block_name + ".html"), 'wb')
                                            # Write the block page
                                            file.write(block_page)
                                            
                                            print("html file saved")
                                            file.close()
                                            
                                    except Exception as e:
                                        print(e)                      
                        except Exception as e:
                            print(e)
        
            except Exception as e:
                print(e)